In [5]:
import json
import os

In [3]:
data_p = '/root/data_alfred/unlabeled_12k_20201206/seen'
toy_splits_p = '/root/data_alfred/splits/debug_20201210.json'
splits_p = '/root/data_alfred/splits/unlabeled_12k_20201206.json'

In [28]:
def truncate_extra_subgoals(traj_data, num_subgoals, key):
    assert len(traj_data[key]['anns'][0]['high_descs']) >= num_subgoals
    if len(traj_data[key]['anns'][0]['high_descs']) > num_subgoals:
        traj_data[key]['anns'][0]['high_descs'] = traj_data[key]['anns'][0]['high_descs'][:num_subgoals]
        return True
    else:
        return False

def match_post_prediction_subgoal_lengths(split, data_p, overwrite_traj=False, debug=False):
    '''
    split: a list of tasks {'task':<task name>/<trial id>, 'repeat_idx':int, 'full_traj_success':boolean, 'collected_subgoals':int}
    '''
    adjusts_explainer_full = 0
    adjusts_explainer = 0
    adjusts_baseline = 0
    
    fail = 0
    for task in split:
        traj_data_p = os.path.join(data_p, task['task'], 'traj_data.json')
        with open(traj_data_p, 'r') as f:
            traj_data = json.load(f)

        if debug:
            print (task)
            print ('Explainer Full')
            print (traj_data['explainer_full_annotations'])
            print('pre truncation')
            print ('Explainer Aux Loss only')
            print (traj_data['explainer_annotations'])
            print ('Baseline')
            print (traj_data['baseline_annotations'])
            print ('----------------------------------------')
            
        true_num_subgoals = len(traj_data['plan']['high_pddl'])
        
        # verify that the predicted instructions has # subgoals >= gold # subgoals
        # when predicting in a batch of different tasks, model can decode more than necessary
        assert len(traj_data['explainer_full_annotations']['anns'][0]['high_descs']) >= true_num_subgoals
        assert len(traj_data['explainer_annotations']['anns'][0]['high_descs']) >= (true_num_subgoals - 1)
        assert len(traj_data['baseline_annotations']['anns'][0]['high_descs']) >= (true_num_subgoals -1)
        
        adjusts_explainer_full += int(truncate_extra_subgoals(traj_data, true_num_subgoals, key='explainer_full_annotations'))
        adjusts_explainer += int(truncate_extra_subgoals(traj_data, true_num_subgoals, key='explainer_annotations'))
        adjusts_baseline += int(truncate_extra_subgoals(traj_data, true_num_subgoals, key='baseline_annotations'))

        if debug:
            print (adjusts_explainer, adjusts_baseline)
            print (task)
            print('post truncation')
            print ('Explainer Full')
            print (traj_data['explainer_full_annotations'])
            print ('Explainer Aux Loss only')
            print (traj_data['explainer_annotations'])
            print ('Baseline')
            print (traj_data['baseline_annotations'])
            print ('\n\n\n\n\n\n\n\n')
        
        # be careful about overwriting!
        if overwrite_traj:
            with open(traj_data_p, 'w') as f:
                json.dump(traj_data, f)
    
    return adjusts_explainer_full, adjusts_explainer, adjusts_baseline
        

## Debug with Toy Split

In [6]:
with open(toy_splits_p, 'r') as f:
    toy_split = json.load(f)['augmentation']

In [29]:
adjusts_explainer_full, adjusts_explainer, adjusts_baseline = match_post_prediction_subgoal_lengths(toy_split, data_p, overwrite_traj=False, debug=True)

{'task': 'pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373', 'repeat_idx': 0, 'full_traj_success': True, 'collected_subgoals': 6}
pre truncation
Explainer Aux Loss only
{'anns': [{'task_desc': 'put a chilled tomato in the sink .', 'high_descs': ['turn around and walk to the sink .', 'pick up the tomato slice from the sink .', 'turn right and walk to the fridge .', 'chill the tomato in the fridge then get it back out and close the door .', 'turn left and walk to the sink then turn right and walk to the sink .', 'put the tomato in the sink .', 'turn right and walk over to the small wooden shelving unit in the corner of the room .', 'turn right towards the small white table .', 'turn right and walk over to the small wooden shelving unit in the corner of the room .', 'turn right and walk over to the small wooden shelving unit in the corner of the room .', 'move forward slightly then turn right towards the skin finish face the sink drawer', 'move forward 

(Pdb) q


BdbQuit: 

In [22]:
fail

34